This is a list of antimicrobial medicines. 

In [2]:
from ebmdatalab import bq
import os
import pandas as pd

## Antimicrobials in BNF

In [5]:
sql = '''
WITH bnf_codes AS (  
  SELECT bnf_code, chemical FROM hscic.bnf WHERE 
  (bnf_code LIKE '0501%')
) 
  
SELECT "vmp" AS type, id, bnf_code, chemical, nm
FROM dmd.vmp
WHERE bnf_code IN (SELECT * FROM bnf_codes)

UNION ALL

SELECT "amp" AS type, id, bnf_code, chemical, descr
FROM dmd.amp
WHERE bnf_code IN (SELECT * FROM bnf_codes)

ORDER BY type, nm '''

antibac_meds = bq.cached_read(sql, csv_path=os.path.join('..','data','antibac_meds.csv'))
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
antibac_meds.count()

C:\Users\hcurtis\Anaconda3\envs\antibiotics-rct\lib\site-packages\pandas_gbq\gbq.py:547: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  df = rows_iter.to_dataframe(


type        3387
id          3387
bnf_code    3387
nm          3387
dtype: int64

## Dm+d Additions
There are some medicines used in hospital only, which don't have BNF codes (used for primary care). We will manually select these from dmd using keywords.

In [9]:
sql = '''
WITH dmd_codes AS (  
  SELECT id FROM dmd.vmp WHERE
  LOWER(nm) LIKE '%micin %'     OR
  LOWER(nm) LIKE '%mycin %'      OR
  LOWER(nm) LIKE '%cycline %'     

  )      
   
SELECT "vmp" AS type, id, bnf_code, nm
FROM dmd.vmp
WHERE id IN (SELECT * FROM dmd_codes)

UNION ALL

SELECT "amp" AS type, id, bnf_code, descr
FROM dmd.amp
WHERE vmp IN (SELECT * FROM dmd_codes)

ORDER BY type, nm '''

dmd_antibac_meds = bq.cached_read(sql, csv_path=os.path.join('..','data','dmd_antibac_meds .csv'))
dmd_antibac_meds.count()

type        1300
id          1300
bnf_code     983
nm          1300
dtype: int64

In [10]:
#join the two datframes
antibacterials = pd.concat([antibac_meds, dmd_antibac_meds])
antibacterials.count()

type        4687
id          4687
bnf_code    4370
nm          4687
dtype: int64

In [5]:
antibacterials.sort_values(["type", "nm"])

,type,id,bnf_code,nm
0,amp,20158911000001108,0801050BEBBAAAA,Abraxane 100mg powder for suspension for infusion vials (Celgene Ltd)
1,amp,14953811000001108,NaN,Abraxane 100mg powder for suspension for infusion vials (Celgene Ltd)
0,amp,20544111000001106,1001030Q0BDAAAB,Acadione 250mg tablets (Imported (France))
1,amp,18044811000001100,0802020T0BEACAF,Adoport 0.5mg capsules (Sandoz Ltd)
2,amp,28800611000001103,0802020T0BEADBF,Adoport 0.75mg capsules (Sandoz Ltd)
3,amp,18044411000001102,0802020T0BEAAAB,Adoport 1mg capsules (Sandoz Ltd)
4,amp,28800911000001109,0802020T0BEAEBG,Adoport 2mg capsules (Sandoz Ltd)
5,amp,18044211000001101,0802020T0BEABAA,Adoport 5mg capsules (Sandoz Ltd)
6,amp,11619211000001106,0802020T0BCAAAM,Advagraf 0.5mg modified-release capsules (Astellas Pharma Ltd)
7,amp,11619611000001108,0802020T0BCABAN,Advagraf 1mg modified-release capsules (Astellas Pharma Ltd)


In [6]:
antibac_meds.to_csv(os.path.join('..','data','antibac_codelist.csv')) #export to csv here
